# 🕒 Medición de Tiempo de Arranque: VM vs Docker

Este notebook mide y compara el tiempo que tarda en arrancar:
- Una máquina virtual (VM) ejecutada en VirtualBox.
- Un contenedor Docker basado en una imagen ligera (por ejemplo, Ubuntu o Alpine).

El objetivo es analizar cuál de los dos entornos se inicializa más rápido.

---


## ⚙️ Cómo funciona el script

El script `medicion_arranque.py` realiza dos tareas principales:

1️⃣ **Arranque de Docker**
- Lanza un contenedor temporal usando la imagen `ubuntu` (o `alpine`).
- Mide el tiempo que tarda el contenedor en ejecutarse y salir.

2️⃣ **Arranque de VM**
- Usa `VBoxManage` (herramienta de línea de comandos de VirtualBox) para iniciar una máquina virtual específica (debes tener VirtualBox instalado).
- Espera a que la máquina esté accesible por red, verificando que responda por el puerto 22 (SSH).

El script guarda los resultados en un archivo llamado:


## 🛠️ Configuración necesaria en VirtualBox

Antes de ejecutar este notebook, asegúrate de:

✅ Tener una máquina virtual creada en VirtualBox.  
✅ Configurar la VM para:
- Asignar una **IP fija** en red interna o puenteada.
- Tener habilitado el servicio SSH (para que responda en el puerto 22).
- Anotar el **nombre exacto** de la VM (usado en VirtualBox, no solo el nombre visible).

Ejemplo de ajustes de red:
- Red → Adaptador puente (para usar red local).
- O Red interna con configuración manual.

Asegúrate también de que puedes hacer ping o conexión SSH desde tu máquina host al IP configurado.

---


In [ ]:
#Configuración de parámetros e importacion
import subprocess
import time
import socket
import os
# Configuración de prueba
VM_NAME = "VM Clase"            # Nombre EXACTO de la VM en VirtualBox
VM_IP = "192.168.56.101"        # IP fija asignada a la VM
VM_PORT = 22                    # Puerto SSH (usualmente 22)
DOCKER_IMAGE = "ubuntu"         # Imagen de Docker para la prueba (puedes usar "alpine")

In [ ]:
#Funciones de medición
# Crear carpeta resultados si no existe
os.makedirs("results", exist_ok=True)

def medir_arranque_docker():
    print("\n🐳 Midiendo arranque de Docker...")
    start = time.time()
    result = subprocess.run([
        "docker", "run", "--rm", DOCKER_IMAGE, "sh", "-c", "echo 'Listo desde Docker'"
    ], capture_output=True, text=True)
    end = time.time()
    print(result.stdout.strip())
    print(f"⏱️ Docker arrancó en {end - start:.2f} segundos")
    return end - start

def esperar_puerto(ip, puerto, timeout=60):
    print(f"🔌 Esperando que la VM esté activa en {ip}:{puerto}...")
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((ip, puerto), timeout=2):
                return True
        except (socket.timeout, ConnectionRefusedError, OSError):
            time.sleep(1)
    return False

def medir_arranque_vm():
    print("\n🖥️ Midiendo arranque de VM...")
    start = time.time()
    subprocess.run(["VBoxManage", "startvm", VM_NAME, "--type", "headless"])
    if esperar_puerto(VM_IP, VM_PORT):
        end = time.time()
        print(f"⏱️ VM arrancó en {end - start:.2f} segundos")
        return end - start
    else:
        print("⚠️ No se pudo comprobar el arranque de la VM (timeout)")
        return None


In [ ]:
#Ejecutar las mediciones
tiempo_docker = medir_arranque_docker()
tiempo_vm = medir_arranque_vm()

# Guardar resultados
with open("resultados/tiempos_arranque.txt", "w") as f:
    f.write("=== Tiempos de Arranque ===\n")
    f.write(f"Docker: {tiempo_docker:.2f} segundos\n")
    if tiempo_vm is not None:
        f.write(f"VM: {tiempo_vm:.2f} segundos\n")
    else:
        f.write("VM: No disponible (falló la conexión)\n")

print("\n✅ Resultados guardados en resultados/tiempos_arranque.txt")


In [ ]:
#Leer resultados y graficar
import matplotlib.pyplot as plt

# Leer resultados del archivo
tiempos = {}
with open("resultados/tiempos_arranque.txt", "r") as f:
    for line in f:
        if "Docker" in line:
            tiempos["Docker"] = float(line.strip().split(": ")[1].split()[0])
        if "VM" in line and "No disponible" not in line:
            tiempos["VM"] = float(line.strip().split(": ")[1].split()[0])

# Graficar
plt.figure(figsize=(6, 4))
plt.bar(tiempos.keys(), tiempos.values(), color=['skyblue', 'orange'])
plt.ylabel('Tiempo (segundos)')
plt.title('Comparación de Tiempo de Arranque\nDocker vs Máquina Virtual')
plt.show()
